In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))
import os

In [2]:
import hail as hl

localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'
hl.init(tmp_dir=localfs_path, local_tmpdir=localfs_path, 
        spark_conf={'spark.driver.memory': '15G', 'spark.executor.memory': '30G'})

2023-02-05 15:34:46.256 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0386:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.105-acd89e80c345
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/analysis/variant-analysis-and-exports/hail-20230205-1534-0.2.105-acd89e80c345.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis
from scipy import stats

output_notebook()

Loading BokehJS ...

Loading BokehJS ...

### Phenotypes to investigate and their respective phenotypes in genebass:


Many of the phenotypes do not have their exact respective phenotypes, so the closest phenotype was chosen:

panukb phenotype | genebass phenotype
--------------- | ------------------
anti_gout_agent_microtuble_polymerization_inhibitor-both_sexes | none
allopurinol | categorical-20003-both_sexes-1140875408- - Treatment/medication code
immature_reticulocyte_fraction | Immature reticulocyte fraction
both_sexes--platelet_crit | Platelet crit
biobankuk-egfrcreacys-both_sexes--estimated_glomerular_filtration_rate_cystain_c-EUR | none
forced vital capacity best measure | fvc best measure
biobankuk-20002-both_sexes-1309-non_cancer_illness_code_self_reported-EUR | M81 Osteoporosis without pathological fracture

- gene lists were prepared from the genebass results SKAT-O tests based on the previously set genome-wide significance threshold: 2.5 × 10−7
- We also conducted analyses for the 1 x 10-6 and 1 x 10-5 thresholds

### Import and filter genebass:

In [4]:
genebass = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/resources/genebass-500k/results.mt')

In [5]:
pheno_filters = (
    genebass.coding == "1140875408"
) | (
    genebass.description == "Immature reticulocyte fraction"
) | (
    genebass.description == "Platelet crit"
) | (
    genebass.description == "Forced vital capacity (FVC), Best measure"
) | (
    genebass.description_more.contains('M81')
)  

In [6]:
genebass = genebass.filter_cols(pheno_filters)

In [7]:
genebass.count()

(75767, 5)

### Get gene lists for different cut offs:

In [8]:
genebass = genebass.annotate_cols(
    below_7 = hl.agg.filter((
        genebass.Pvalue < 2.5e-07),
        hl.agg.collect_as_set(genebass.gene_symbol)
    ),
    below_6 = hl.agg.filter((
        genebass.Pvalue < 1e-06),
        hl.agg.collect_as_set(genebass.gene_symbol)
    ),
    below_5 = hl.agg.filter((
        genebass.Pvalue < 1e-05),
        hl.agg.collect_as_set(genebass.gene_symbol)
    )
)

In [9]:
genebass = genebass.select_cols(
    genebass.below_7,
    genebass.below_6,
    genebass.below_5,
    genebass.description,
    genebass.description_more,
    genebass.coding_description
)

In [10]:
genebass = genebass.annotate_cols(
    len_below_5 = hl.len(genebass.below_5),
    len_below_6 = hl.len(genebass.below_6),
    len_below_7 = hl.len(genebass.below_7)
)

In [47]:
genebass = genebass.annotate_cols(
    description = hl.coalesce(
        genebass.coding_description,
        genebass.description
    )
)

### The matrix table below has already been filtered and annotated, see preprocessing/joint-with-gts/genotype-and-filter.ipynb

In [11]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/joint/full-healthy.mt')

In [12]:
mt.count()

(2639210, 237)

In [13]:
proper_controls = ['WGS_37b', 'WGS_37c', 'WGS_163d', 'WGS_7120', 'WGS_7142', 'WGS_7143', 'WGS_7152',
 'WGS_7153', 'WGS_85b', 'WGS_147c', 'WGS_180b', 'WGS_185c', 'WGS_6819', 'S_7213', 'S_7227', 'S_7241', 'S_7246', 'S_7254', 'S_7274',
                   'S_7307', '494', '462', '468', '492', '490'] 

In [14]:
len(proper_controls)

25

### Only keep intragenic variants

In [15]:
mt = mt.filter_rows(mt.within_gene == hl.empty_array(hl.tstr), keep = False)
mt.write(localfs_path+'intragenic.mt', overwrite = True)

2023-02-05 15:41:27.361 Hail: INFO: wrote matrix table with 1683507 rows and 237 columns in 14999 partitions to /localfs/1769963/intragenic.mt


In [17]:
mt = hl.read_matrix_table(localfs_path+'intragenic.mt')

In [18]:
mt.count()

(1683507, 237)

### Only keep proprerly healthy controls

In [19]:
mt.aggregate_cols(hl.agg.counter(mt.group))

{'1kg': 98, 'control': 39, 'sport': 100}

In [20]:
mt = mt.filter_cols(
    (mt.group == '1kg') | (mt.group == 'sport') | (hl.literal(proper_controls)).contains(mt.s)
)

In [21]:
mt.write(localfs_path+'filtered.mt', overwrite = True)
mt = hl.read_matrix_table(localfs_path+'filtered.mt')

2023-02-05 15:46:23.629 Hail: INFO: wrote matrix table with 1683507 rows and 220 columns in 14999 partitions to /localfs/1769963/filtered.mt


In [22]:
mt.count()

(1683507, 220)

In [23]:
mt.naive_coalesce(200).write(localfs_path+'repartitioned.mt', overwrite = True)

2023-02-05 15:47:43.882 Hail: INFO: wrote matrix table with 1683507 rows and 220 columns in 200 partitions to /localfs/1769963/repartitioned.mt


In [24]:
mt = hl.read_matrix_table(localfs_path+'repartitioned.mt')

## calculate per-gene burden in 5 ways:
 - gene has a variant with CADD > 20
 - gene has a variant with CADD > 16
 - sum of CADD for variants with CADD > 10
 - LoF - HC lof from vep 
 - gene has a variant with MAF < 0.002 in a database of Polish genomes of has not been found there
 

In [25]:
mt = mt.explode_rows(mt.within_gene)
mt.write(localfs_path+'exploded.mt', overwrite = True)

2023-02-05 15:49:07.424 Hail: INFO: wrote matrix table with 1915897 rows and 220 columns in 200 partitions to /localfs/1769963/exploded.mt


In [26]:
mt = hl.read_matrix_table(localfs_path+'exploded.mt')

In [27]:
mt = mt.filter_cols(mt.s != '494')

In [28]:
is_lof = (mt.vep.vep.transcript_consequences.lof.contains('HC'))

In [29]:
mt = mt.annotate_entries(
    cadd_above_10_value = hl.if_else(
        (mt.cadd.cadd_score > 10),
        (mt.GT.is_non_ref()*mt.cadd.cadd_score),
        hl.int(0)
    ),
    cadd_above_20 = hl.if_else(
        (mt.cadd.cadd_score > 20),
        hl.int(mt.GT.is_non_ref()),
        hl.int(0)
    ),
    cadd_above_16 = hl.if_else(
        (mt.cadd.cadd_score > 16),
        hl.int(mt.GT.is_non_ref()),
        hl.int(0)
    ),
    lof_variant = hl.if_else(
        is_lof,
        hl.int(mt.GT.is_non_ref()),
        hl.int(0)
    ),
    rare_polish_variant = hl.if_else(
        hl.is_defined(mt.polish_af.AF[0]),
        hl.int((mt.polish_af.AF[0] < 0.001)*mt.GT.is_non_ref()),
        hl.int(1)
    )
)

In [30]:
mt.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden.mt', overwrite = True)

2023-02-05 15:51:28.683 Hail: INFO: wrote matrix table with 1915897 rows and 219 columns in 200 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden.mt


In [31]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden.mt')

In [32]:
# group by gene and aggregate
per_gene = mt.group_rows_by(
    mt.within_gene).aggregate(
    cadd_above_10_sum = hl.agg.sum(mt.cadd_above_10_value),
    cadd_above_20_any = hl.int(hl.agg.any(mt.cadd_above_20 == 1)),
    cadd_above_16_any = hl.int(hl.agg.any(mt.cadd_above_16 == 1)),
    lof_any = hl.int(hl.agg.any(mt.lof_variant == 1)),
    rare_polish_any = hl.int(hl.agg.any(mt.rare_polish_variant == 1))
)

In [33]:
per_gene = per_gene.naive_coalesce(1)
per_gene.write(localfs_path+'per_gene.mt', overwrite = True)

2023-02-05 15:51:58.430 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-02-05 15:52:31.129 Hail: INFO: wrote matrix table with 36346 rows and 219 columns in 1 partition to /localfs/1769963/per_gene.mt


In [34]:
per_gene = hl.read_matrix_table(localfs_path+'per_gene.mt')

In [35]:
per_gene.count()

(36346, 219)

In [36]:
per_gene = per_gene.filter_rows(
    hl.agg.any(
        (per_gene.cadd_above_10_sum > 0
        ) | (
        per_gene.cadd_above_20_any == 1
        ) | (
        per_gene.cadd_above_16_any == 1
        ) | (
        per_gene.lof_any == 1
        ) | (
        per_gene.rare_polish_any == 1
        )
    )  
)

In [37]:
per_gene.count()

(27416, 219)

In [38]:
# check per_sample/per group values of burden. Are there any irregulaties overall?

per_gene = per_gene.annotate_cols(
    cadd_above_10_s = hl.agg.stats(per_gene.cadd_above_10_sum),
    cadd_above_20_s = hl.agg.stats(per_gene.cadd_above_20_any),
    cadd_above_16_s = hl.agg.stats(per_gene.cadd_above_16_any),
    lof_any_s = hl.agg.stats(per_gene.lof_any),
    rare_polish_s = hl.agg.stats(per_gene.rare_polish_any)
)

### Perform PCA on variant burden across cohorts

The option - rare polish has been discarded from now on as no variants passed this filter

In [39]:
eigenvalues_10, scores_10, _ = hl.pca(
    per_gene.cadd_above_10_sum,
    k=2
)
eigenvalues_20, scores_20, _ = hl.pca(
    per_gene.cadd_above_20_any,
    k=2
)
eigenvalues_16, scores_16, _ = hl.pca(
    per_gene.cadd_above_16_any,
    k=2
)
eigenvalues_lof, scores_lof, _ = hl.pca(
    per_gene.lof_any,
    k=2
)
eigenvalues_rare, scores_rare, _ = hl.pca(
    per_gene.rare_polish_any,
    k=2
)

2023-02-05 15:52:32.805 Hail: INFO: pca: running PCA with 2 components...
2023-02-05 15:52:36.989 Hail: INFO: pca: running PCA with 2 components...1) / 1]
2023-02-05 15:52:39.293 Hail: INFO: pca: running PCA with 2 components...
2023-02-05 15:52:42.122 Hail: INFO: pca: running PCA with 2 components...1) / 1]
2023-02-05 15:52:45.661 Hail: INFO: pca: running PCA with 2 components...


In [51]:
per_gene = per_gene.annotate_cols(
    pca_10 = scores_10[per_gene.s].scores,
    pca_20 = scores_20[per_gene.s].scores,
    pca_16 = scores_16[per_gene.s].scores,
    pca_lof = scores_lof[per_gene.s].scores,
    pca_rare = scores_rare[per_gene.s].scores
)

In [41]:
p = hl.plot.scatter(per_gene.pca_10[0],
                    per_gene.pca_10[1],
                    label=per_gene.group,
                    title='PCA - sum of variants with CADD over 10', xlabel='PC1', ylabel='PC2')
show(p)

In [42]:
p2 = hl.plot.scatter(per_gene.pca_20[0],
                    per_gene.pca_20[1],
                    label=per_gene.group,
                    title='PCA - any variants with CADD over 20', xlabel='PC1', ylabel='PC2')
show(p2)

In [43]:
p3 = hl.plot.scatter(per_gene.pca_16[0],
                    per_gene.pca_16[1],
                    label=per_gene.group,
                    title='PCA - any variants with CADD over 16', xlabel='PC1', ylabel='PC2')
show(p3)

In [44]:
p4 = hl.plot.scatter(per_gene.pca_lof[0],
                     per_gene.pca_10[1],
                     label=per_gene.group,
                     title='PCA - any LoF variants', xlabel='PC1', ylabel='PC2')
show(p4)

In [45]:
p5 = hl.plot.scatter(per_gene.pca_rare[0],
                     per_gene.pca_rare[1],
                     label=per_gene.group,
                     title='PCA - any variants rare in Polish population', xlabel='PC1', ylabel='PC2')
show(p5)

### Annotate the per gene table with phenotypes:

In [56]:
gb = genebass.cols()

In [56]:
per_gene = per_gene.index_rows(per_gene.within_gene)

AttributeError: StructExpression instance has no field, method, or property 'index_rows'
    Hint: use 'describe()' to show the names of all data fields.

In [55]:
options = ['below_7','below_6','below_5']

for o in options:
    res = gb.explode(gb[o])
    res = res.index(res[o])
    per_gene = per_gene.annotate_rows(
       o = res.index(per_gene.rows()['within_gene'], all_matches = True)['description']
    )

ExpressionException: Key type mismatch: cannot index table with given expressions:
  Table key:         str, str, str, str, str
  Index Expressions: str

In [ ]:
genes.annotate(hpo = hpo.index(genes['hg38.kgXref.geneSymbol'], all_matches = True)['f1'])

genebass = genebass.select_cols(
    genebass.below_7,
    genebass.below_6,
    genebass.below_5,
    genebass.description,
    genebass.description_more,
    genebass.coding_description
)

per_gene = per_gene.annotate_rows(
    phenos_below_7 = hl.,
    phenos_below_6 = ,
    phenos_below_5 = 

In [ ]:
genebass = genebass.select_cols(
    genebass.below_7,
    genebass.below_6,
    genebass.below_5,
    genebass.description,
    genebass.description_more,
    genebass.coding_description
)

In [ ]:
burden_lof = mt.filter_rows(mt['gnomad_v_3_1']['vep']['transcript_consequences']['lof'].contains('HC')) #option 2
burden_lof.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_lof.mt')

In [ ]:
b_cadd = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_cadd.mt')
b_lof = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/for_burden_lof.mt')

In [ ]:
per_gene_cadd = b_cadd.group_rows_by(b_cadd.within_gene).aggregate(
    burden_cadd_phred = hl.agg.sum(b_cadd.cadd_entry)
)

per_gene_lof = b_lof.group_rows_by(b_lof.within_gene).aggregate(
    lof = hl.agg.sum(b_lof.GT.n_alt_alleles()
                    )
)

In [ ]:
per_gene_cadd.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_cadd.mt')
per_gene_lof.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_lof.mt')

### Calculate burden statistics

In [4]:
per_gene_cadd = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_cadd.mt')
per_gene_lof = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-sportwgs/data/hail-mts/per_gene_burden_lof.mt')

In [5]:
per_gene_cadd.count()

(7432, 219)

In [6]:
per_gene_lof.count()

(1018, 219)

In [70]:
# annotate per_gene lof and cadd table with gene lists
phenos = {
    'gout':hl.set(["ABCG2", "ADH1C"]),
    'retic':hl.set(["CALR", "R3HDM4", "RNF10", "TLN2",
         "IFRD2", "GMPR", "ENG", "EIF2AK1"]),
    'crit':hl.set(["CALR", "JAK2", "THPO", "CHEK2",
        "TUBB1", "LY75", "PTPRH", "LY75-CD302",
        "MPL", "TET2"]),
    'fvc':hl.set(["SMAD6", "ADAMTSL4", "DDR2", "IRS1"]),
    'fev':hl.set(["KDM5B","ADAMTSL4","ANKRD12","PSMD13","SUPT6H","RIF1","TNRC6B","IRS1"])
}

pgc = per_gene_cadd.annotate_rows(**{phen:
                                     phenos[phen].contains(per_gene_cadd.within_gene)
                                     for phen in phenos.keys()})
pgc = pgc.filter_rows(
    hl.any(*[pgc[phen] for phen in phenos.keys()])
)
pgc = pgc.naive_coalesce(1)
pgc = pgc.write(localfs_path+'pgc.ht')


pgl = per_gene_lof.annotate_rows(**{phen:
                                     phenos[phen].contains(per_gene_lof.within_gene)
                                     for phen in phenos.keys()})
pgl = pgl.filter_rows(
    hl.any(*[pgl[phen] for phen in phenos.keys()])
)
pgl = pgl.naive_coalesce(1)
pgl = pgl.write(localfs_path+'pgl.ht') #there are not lof genes in the lists, so this is empty

2023-01-27 11:36:01.702 Hail: INFO: wrote matrix table with 13 rows and 219 columns in 1 partition to /localfs/1726769/pgc.ht
2023-01-27 11:36:28.114 Hail: INFO: wrote matrix table with 0 rows and 219 columns in 1 partition to /localfs/1726769/pgl.ht


In [99]:
pgc = hl.read_matrix_table(localfs_path+'pgc.ht')

In [98]:
pgc.rows().show()

,,,,,
within_gene,gout,retic,crit,fvc,fev
str,bool,bool,bool,bool,bool
"""ADAMTSL4""",False,False,False,True,True
"""EIF2AK1""",False,True,False,False,False
"""ENG""",False,True,False,False,False
"""GMPR""",False,True,False,False,False
"""KDM5B""",False,False,False,False,True
"""LY75""",False,False,True,False,False
"""LY75-CD302""",False,False,True,False,False
"""PSMD13""",False,False,False,False,True


In [156]:
type(test[test['type'] == 'speed']['burden_score'])

pandas.core.series.Series

In [168]:
ag_pds = {}
phenos_list = ['retic', 'crit', 'fvc', 'fev']
# I skip gout because there is no burden there in any sample

def ttest(df, x, y, field):
    pval = stats.ttest_ind(
        list(pd.to_numeric(df[df[field] == x]['burden_score'])),
        list(pd.to_numeric(df[df[field] == y]['burden_score']))
    ).pvalue
    return pval

In [169]:
for phen in phenos_list:
    test = pgc.filter_rows(pgc[phen])
    test = test.annotate_cols(
        burden_score = hl.agg.sum(test['burden_cadd_phred'])
    )
    test = test.select_cols('group', 'burden_score', type=test.sport_phenotypes.type)
    test = test.cols()
    test = test.to_pandas()
    test = test.drop('s', axis=1)
    test['type'] = test[['type', 'group']].bfill(axis=1).iloc[:, 0]
    print('phenotype: '+phen)
    print('controls GTS mean:')
    print(test[test['group'] == 'control']['burden_score'].mean())
    print('sportsmen mean:')
    print(test[test['group'] == 'sport']['burden_score'].mean())
    print('controls 1kg mean:')
    print(test[test['group'] == '1kg']['burden_score'].mean())
    print('speed mean:')
    print(test[test['type'] == 'speed']['burden_score'].mean())
    print('endurance mean:')
    print(test[test['type'] == 'endurance']['burden_score'].mean())
    print('endurance vs speed t-test p value:')
    print(ttest(test, 'endurance', 'speed', 'type'))
    ag_pds[phen] = test

2023-01-27 13:00:55.074 Hail: INFO: Coerced sorted dataset


phenotype: retic
controls GTS mean:
1.180952380952381
sportsmen mean:
3.8440000000000003
controls 1kg mean:
2.287755102040816
speed mean:
4.590196078431372
endurance mean:
3.0673469387755103
endurance vs speed t-test p value:
0.4453511741655626


2023-01-27 13:00:55.656 Hail: INFO: Coerced sorted dataset


phenotype: crit
controls GTS mean:
5.4714285714285715
sportsmen mean:
8.792
controls 1kg mean:
9.350000000000001
speed mean:
10.445098039215686
endurance mean:
7.071428571428573
endurance vs speed t-test p value:
0.2255788931568009


2023-01-27 13:00:56.241 Hail: INFO: Coerced sorted dataset


phenotype: fvc
controls GTS mean:
2.9000000000000004
sportsmen mean:
4.518
controls 1kg mean:
3.107142857142857
speed mean:
3.270588235294118
endurance mean:
5.816326530612245
endurance vs speed t-test p value:
0.16284433774761367


2023-01-27 13:00:56.846 Hail: INFO: Coerced sorted dataset


phenotype: fev
controls GTS mean:
0.0
sportsmen mean:
1.287
controls 1kg mean:
0.47040816326530605
speed mean:
1.4862745098039218
endurance mean:
1.079591836734694
endurance vs speed t-test p value:
0.7229238410597287
